In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import re
import glob

from my_utils import *

In [4]:
os.environ["FSLDIR"] = os.environ["CONDA_PREFIX"]
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"  #  default output type for fslmaths 

# fslmaths wants this. It sets some variables in the environment
#export FSLDIR=$CONDA_PREFIX
#source $FSLDIR/etc/fslconf/fsl.sh # check this fie

# in %%bash magic cell does not work. Creates a new temporary 
# Bash subprocess for that cell.  the subprocess terminates, and any export statements are lost.
# we can do it in conda environment at activation time 
# for now we just do (those are inherited by the subprocesses)


In [5]:
# Auxiliary Parameter files used for registration
non_param_file = "aux-01-reg/nonlinearRegistration.param"
standard_reference = "aux-01-reg/standard_ref.nii.gz"
STANDARD_RSP_REFERENCE = "aux-01-reg/standard_ref_rsp.nii.gz"
DIS_ATLAS_MASK_FILE = "aux-01-reg/dis_atlas_mask.nii.gz"

# OUTPUT FOLDERS The authors first delete them before creation
os.makedirs("xform", exist_ok=True)
os.makedirs("registered", exist_ok=True)
os.makedirs("divided/C0divC1", exist_ok=True)
os.makedirs("divided/C2divC1", exist_ok=True) # only used if C2 is present
os.makedirs("jacobian", exist_ok=True)


# logs directory (we dont need)
#os.makedirs("scripts", exist_ok=True) 
#os.makedirs("logs", exist_ok=True)


In [6]:
# Preprocessing
input_pattern = "nrrd/*.nrrd" # all nrrd files in the nrrd/ folder
for file in glob.glob(input_pattern):
    #print(f"Processing file: {file} to {file.replace('.nrrd', '__optfixed.nii.gz')}")
    out = file.replace('.nrrd', '__optfixed.nii.gz')
    out = out.replace('nrrd/', 'optfixed/')
    #print(f'Processing {file} --> {out}')
    command = f"biswebnode preprocessoptical --ras --biascorrect false --mask false --normalize false --resample --factor 1 --sigma 1 -i {file} --output {out}"
    #print(command)
    #run_command(command)


In [7]:

# CHECK your files in optfixed directory (block if some files are missing)
# * is a wildcard for any string
geno_types = ["wt", "het", "hom"]
for geno_type in geno_types:
    check_files(f"optfixed/*_{geno_type}_*C0__optfixed.nii.gz", list = False) # list = True to see the files
    check_files(f"optfixed/*_{geno_type}_*C1__optfixed.nii.gz", list = False)

7 Files found for pattern: optfixed/*_wt_*C0__optfixed.nii.gz
7 Files found for pattern: optfixed/*_wt_*C1__optfixed.nii.gz
9 Files found for pattern: optfixed/*_het_*C0__optfixed.nii.gz
9 Files found for pattern: optfixed/*_het_*C1__optfixed.nii.gz
10 Files found for pattern: optfixed/*_hom_*C0__optfixed.nii.gz
10 Files found for pattern: optfixed/*_hom_*C1__optfixed.nii.gz


In [8]:
# HERE THE AUTOHORs USE SLURM (a job for each file)
# biswebnode seems to run in single thread. We can run in parallel using python. Each file is independent

# Store commands to run in parallel
ls_com1 = []  # Nonlinear Registration
ls_com2 = []  # Reslicing 
ls_com3 = []  # Reslicing 
ls_com4 = []  # Dividing

# We just use C0 to take all file names
input_pattern = f"optfixed/*C0__optfixed.nii.gz" 
for file_path in sorted(glob.glob(input_pattern)):
    # Extract the file name without path and extension
    # example:  optfixed/dyrk1a_exp1_het_8_C0__optfixed.nii.gz --> dyrk1a_exp1_het_8
    file_base = os.path.basename(file_path)
    file = file_base.replace("_C0__optfixed.nii.gz", "")
    #print(file_path + " --> " + file)

    #print(f"Processing file: {file}")
    input_c0_file = f"optfixed/{file}_C0__optfixed.nii.gz"
    input_c1_file = f"optfixed/{file}_C1__optfixed.nii.gz"
    input_c2_file = f"optfixed/{file}_C2__optfixed.nii.gz"

    registered_c0_file = f"registered/r_{file}_C0__optfixed.nii.gz"
    registered_c1_file = f"registered/r_{file}_C1__optfixed.nii.gz"
    registered_c2_file = f"registered/r_{file}_C2__optfixed.nii.gz"

    bisxform_file = f"xform/standard_ref__{file}_C1__optfixed__nlr.bisxform" 

    #if bisxform_file already exists, skip to next file
    if os.path.isfile(bisxform_file):
        print(f"Skipping {file} as bisxform file already exists")
        continue
    else:
        print(f"Processing {file}")

    # Creates registered (C0, C1) and bisxform files
    command = f"biswebnode nonlinearregistration --paramfile {non_param_file} -r {standard_reference} -t {input_c1_file} -o {bisxform_file}" # THIS HEAVY - ~13H per files
    ls_com1.append(command)
    #run_command(command) # Here to run each command in sequence

    command = f"biswebnode resliceImage -r {standard_reference} -i {input_c1_file} -x {bisxform_file} -o {registered_c1_file}"
    ls_com2.append(command)
    #run_command(command)

    command = f"biswebnode resliceImage -r {registered_c1_file} -i {input_c0_file} -x {bisxform_file} -o {registered_c0_file}"
    ls_com3.append(command)
    #run_command(command)

    if os.path.isfile(input_c2_file):
        command = f"biswebnode resliceImage -r {registered_c1_file} -i {input_c2_file} -x {bisxform_file} -o {registered_c2_file}"
        ls_com3.append(command)
        #run_command(command)


    # Creates C0divC1 divided files
    divided_c0_file = f"divided/C0divC1/r_{file}_C0divC1.nii.gz"
    command = f"fslmaths {registered_c0_file} -div {registered_c1_file} {divided_c0_file}"
    ls_com4.append(command)
    #run_command(command)

    if os.path.isfile(input_c2_file):
        # Creates C2divC1 divided files
        divided_c2_file = f"divided/C2divC1/r_{file}_C2divC1.nii.gz"
        command = f"fslmaths {registered_c2_file} -div {registered_c1_file} {divided_c2_file}"
        ls_com4.append(command)
        #run_command(command)

# Run commands in parallel
#run_commands_parallel(ls_com1) # first all the registrations have to be done
#run_commands_parallel(ls_com2)
#run_commands_parallel(ls_com3)
#run_commands_parallel(ls_com4)
    
    

Skipping dyrk1a_exp1_het_1 as bisxform file already exists
Skipping dyrk1a_exp1_het_2 as bisxform file already exists
Skipping dyrk1a_exp1_het_3 as bisxform file already exists
Skipping dyrk1a_exp1_het_4 as bisxform file already exists
Skipping dyrk1a_exp1_het_5 as bisxform file already exists
Skipping dyrk1a_exp1_het_6 as bisxform file already exists
Skipping dyrk1a_exp1_het_7 as bisxform file already exists
Skipping dyrk1a_exp1_het_8 as bisxform file already exists
Skipping dyrk1a_exp1_het_9 as bisxform file already exists
Skipping dyrk1a_exp1_hom_10 as bisxform file already exists
Skipping dyrk1a_exp1_hom_1 as bisxform file already exists
Skipping dyrk1a_exp1_hom_2 as bisxform file already exists
Skipping dyrk1a_exp1_hom_3 as bisxform file already exists
Skipping dyrk1a_exp1_hom_4 as bisxform file already exists
Skipping dyrk1a_exp1_hom_5 as bisxform file already exists
Skipping dyrk1a_exp1_hom_6 as bisxform file already exists
Skipping dyrk1a_exp1_hom_7 as bisxform file already exi

In [10]:
ls_com1 = [] # Jacobian
ls_com2 = [] # Mask Jacobian

for file_path in sorted(glob.glob(input_pattern)):
    file_base = os.path.basename(file_path)
    file = file_base.replace("_C0__optfixed.nii.gz", "")

    bisxform_file = f"xform/standard_ref__{file}_C1__optfixed__nlr.bisxform" # definito sopra

    # Creates jacobian files (ja_ and mask_ja) for C1 using bisxform files
    jacobian_file = f"jacobian/ja_{file}.nii.gz"
    mask_file = f"jacobian/mask_ja_{file}.nii.gz"
    

    if os.path.isfile(mask_file):
        print(f"Skipping {file} as mask_file already exist")
        continue
    else:
        print(f"Processing {file}")

    command1 = f"biswebnode jacobianimage -x {bisxform_file} -i {STANDARD_RSP_REFERENCE} -o {jacobian_file}"
    command2 = f"biswebnode maskimage -i {jacobian_file} -m {DIS_ATLAS_MASK_FILE} -o {mask_file}" 
    #run_command(command1)
    #run_command(command2)
    ls_com1.append(command1)
    ls_com2.append(command2)

run_commands_parallel(ls_com1)
run_commands_parallel(ls_com2)


Skipping dyrk1a_exp1_het_1 as mask_file already exist
Skipping dyrk1a_exp1_het_2 as mask_file already exist
Skipping dyrk1a_exp1_het_3 as mask_file already exist
Skipping dyrk1a_exp1_het_4 as mask_file already exist
Skipping dyrk1a_exp1_het_5 as mask_file already exist
Skipping dyrk1a_exp1_het_6 as mask_file already exist
Skipping dyrk1a_exp1_het_7 as mask_file already exist
Skipping dyrk1a_exp1_het_8 as mask_file already exist
Skipping dyrk1a_exp1_het_9 as mask_file already exist
Skipping dyrk1a_exp1_hom_10 as mask_file already exist
Skipping dyrk1a_exp1_hom_1 as mask_file already exist
Skipping dyrk1a_exp1_hom_2 as mask_file already exist
Skipping dyrk1a_exp1_hom_3 as mask_file already exist
Skipping dyrk1a_exp1_hom_4 as mask_file already exist
Skipping dyrk1a_exp1_hom_5 as mask_file already exist
Skipping dyrk1a_exp1_hom_6 as mask_file already exist
Skipping dyrk1a_exp1_hom_7 as mask_file already exist
Skipping dyrk1a_exp1_hom_8 as mask_file already exist
Skipping dyrk1a_exp1_hom_9 